In [2]:
#-*- coding:utf-8-*- 
import xml.dom
import xml.dom.minidom
import os
# from PIL import Image
import cv2
import json

# xml文件规范定义
_IMAGE_PATH = "/media/yyp/DATA/Public_Datasets/BITVehicle/JPEGImages/"

_INDENT = '' * 4
_NEW_LINE = '\n'
_FOLDER_NODE = 'BITVehicle_Dataset'
_ROOT_NODE = 'annotation'
_DATABASE_NAME = 'LOGODection'
_ANNOTATION = 'PASCAL VOC2007'
_AUTHOR = 'SyGoing_CSDN'
_SEGMENTED = '0'
_DIFFICULT = '0'
_TRUNCATED = '0'
_POSE = 'Unspecified'

 
# _IMAGE_COPY_PATH= 'JPEGImages'
_ANNOTATION_SAVE_PATH = "/media/yyp/DATA/Public_Datasets/BITVehicle/Annotations/"

# _IMAGE_CHANNEL= 3
def createElementNode(doc, tag, attr):  #创建一个元素节点
    element_node = doc.createElement(tag)
 
    # 创建一个文本节点
    text_node = doc.createTextNode(attr)
 
    # 将文本节点作为元素节点的子节点
    element_node.appendChild(text_node)
 
    return element_node
 
# 封装添加一个子节点
def createChildNode(doc, tag, attr, parent_node):
    child_node = createElementNode(doc,tag, attr)
 
    parent_node.appendChild(child_node)
 
 
# object节点比较特殊
clsid_to_midname = {0:'bg', 1:'vehicle', 2:'plate'}
def createObjectNode(doc, attrs):
    object_node =doc.createElement('object')
 
    cls_id = attrs['category_id']
    midname = clsid_to_midname[cls_id]
    print(midname)
    if(midname == 'vehicle'):
        midname = 'car'
 
    createChildNode(doc, 'name', midname,
                    object_node)
 
    #createChildNode(doc, 'name',attrs['name'],
    #                object_node)
 
    createChildNode(doc, 'pose',
                    _POSE, object_node)
 
    createChildNode(doc, 'truncated',
                    _TRUNCATED,object_node)
 
    createChildNode(doc, 'difficult',
                    _DIFFICULT,object_node)
 
    bndbox_node = doc.createElement('bndbox')
 
    createChildNode(doc, 'xmin',str(int(attrs['bbox'][0])),
                    bndbox_node)
 
    createChildNode(doc, 'ymin',str(int(attrs['bbox'][1])),
                    bndbox_node)
 
    createChildNode(doc, 'xmax',str(int(attrs['bbox'][0] + attrs['bbox'][2])),
                    bndbox_node)
 
    createChildNode(doc, 'ymax',str(int(attrs['bbox'][1] + attrs['bbox'][3])),
                    bndbox_node)
 
    object_node.appendChild(bndbox_node)
 
    return object_node
 
# 将documentElement写入XML文件
def writeXMLFile(doc, filename):
    tmpfile = open('tmp.xml', 'w')
 
    doc.writexml(tmpfile, addindent='' *4, newl='\n', encoding='utf-8')
 
 
    tmpfile.close()
 
    # 删除第一行默认添加的标记
 
    fin = open('tmp.xml')
    # print(filename)
    fout = open(filename, 'w')
    # print(os.path.dirname(fout))
 
    lines = fin.readlines()
 
    for line in lines[1:]:
 
        if line.split():
            fout.writelines(line)
 
            # new_lines =''.join(lines[1:])
 
        # fout.write(new_lines)
 
    fin.close()
 
    fout.close()
 
 
if __name__ == "__main__":
    ##读取图片列表
    targetJason = "/media/yyp/DATA/Public_Datasets/BITVehicle/bitvehicle_test.json"
    img_path = _IMAGE_PATH
    fileList = os.listdir(img_path)
    fileList = sorted(fileList)
    if fileList == 0:
        os._exit(-1)
 
    with open(targetJason, "r") as f:
        ann_data = json.load(f)
        #print(type(ann_data))
        #print(len(ann_data))
        #print(ann_data)
        for key in ann_data:
            print(key)
            #print(ann_data[key])
 
    current_dirpath =os.path.dirname(os.path.abspath('__file__'))
 
    if not os.path.exists(_ANNOTATION_SAVE_PATH):
        os.mkdir(_ANNOTATION_SAVE_PATH)
 
        # if notos.path.exists(_IMAGE_COPY_PATH):
    #    os.mkdir(_IMAGE_COPY_PATH)
 
    for imageName in fileList:
        #print(imageName)
        if (not imageName.endswith('.jpg')):
            continue
        saveName =imageName.strip(".jpg")
        #print('saveName：' + saveName)
        # pos =fileList[xText].rfind(".")
        # textName =fileList[xText][:pos]
 
        # ouput_file = open(_TXT_PATH +'/' + fileList[xText])
        # ouput_file =open(_TXT_PATH)
 
        # lines = ouput_file.readlines()
 
        xml_file_name =os.path.join(_ANNOTATION_SAVE_PATH, (saveName + '.xml'))
        # withopen(xml_file_name,"w") as f:
        #     pass
 
        img =cv2.imread(os.path.join(img_path, imageName))
        print(os.path.join(img_path,imageName))
        # cv2.imshow(img)
        height, width, channel =img.shape
        #print(height, width, channel)
 
        my_dom = xml.dom.getDOMImplementation()
 
        doc = my_dom.createDocument(None, _ROOT_NODE, None)
 
        # 获得根节点
        root_node = doc.documentElement
 
        # folder节点
 
        createChildNode(doc, 'folder',_FOLDER_NODE, root_node)
 
        # filename节点
 
        createChildNode(doc, 'filename',saveName + '.jpg', root_node)
 
        # source节点
 
        source_node =doc.createElement('source')
 
        # source的子节点
 
        createChildNode(doc, 'database',_DATABASE_NAME, source_node)
 
        createChildNode(doc, 'annotation',_ANNOTATION, source_node)
 
        createChildNode(doc, 'image','flickr', source_node)
 
        createChildNode(doc, 'flickrid','NULL', source_node)
 
        root_node.appendChild(source_node)
 
        # owner节点
 
        owner_node = doc.createElement('owner')
 
        # owner的子节点
 
        createChildNode(doc, 'flickrid','NULL', owner_node)
 
        createChildNode(doc, 'name',_AUTHOR, owner_node)
 
        root_node.appendChild(owner_node)
 
        # size节点
 
        size_node =doc.createElement('size')
 
        createChildNode(doc, 'width', str(width), size_node)
 
        createChildNode(doc, 'height', str(height), size_node)
 
        createChildNode(doc, 'depth', str(channel), size_node)
 
        root_node.appendChild(size_node)
 
        # segmented节点
 
        createChildNode(doc, 'segmented',_SEGMENTED, root_node)
        keys = ['categories', 'images', 'annotations']
        ann_categories = ann_data['categories']
        ann_images = ann_data['images']
        ann_annotations = ann_data['annotations']
        FLAG_FOUND = False
        for ann in ann_annotations:
            imgName_id = ann["image_id"]
            imgName = 'vehicle_' + str(imgName_id+1).zfill(7)            
            if (saveName == imgName):
                # object节点
                print('imagename:' + imgName)
                object_node =createObjectNode(doc, ann)
                root_node.appendChild(object_node)
                FLAG_FOUND = True
 
            else:
                continue
 
                # 构建XML文件名称
 
        print(xml_file_name)
 
        # 创建XML文件
 
        # createXMLFile(attrs, width,height, xml_file_name)
 
        # # 写入文件
        if(not FLAG_FOUND):
            continue
        writeXMLFile(doc, xml_file_name)

categories
annotations
images
/media/yyp/DATA/Public_Datasets/BITVehicle/JPEGImages/vehicle_0000005.jpg
imagename:vehicle_0000005
vehicle
imagename:vehicle_0000005
plate
imagename:vehicle_0000005
vehicle
/media/yyp/DATA/Public_Datasets/BITVehicle/Annotations/vehicle_0000005.xml
/media/yyp/DATA/Public_Datasets/BITVehicle/JPEGImages/vehicle_0000006.jpg
imagename:vehicle_0000006
vehicle
imagename:vehicle_0000006
plate
/media/yyp/DATA/Public_Datasets/BITVehicle/Annotations/vehicle_0000006.xml
/media/yyp/DATA/Public_Datasets/BITVehicle/JPEGImages/vehicle_0000010.jpg
imagename:vehicle_0000010
vehicle
imagename:vehicle_0000010
plate
/media/yyp/DATA/Public_Datasets/BITVehicle/Annotations/vehicle_0000010.xml
/media/yyp/DATA/Public_Datasets/BITVehicle/JPEGImages/vehicle_0000013.jpg
imagename:vehicle_0000013
vehicle
imagename:vehicle_0000013
plate
/media/yyp/DATA/Public_Datasets/BITVehicle/Annotations/vehicle_0000013.xml
/media/yyp/DATA/Public_Datasets/BITVehicle/JPEGImages/vehicle_0000015.jpg
im

/media/yyp/DATA/Public_Datasets/BITVehicle/JPEGImages/vehicle_0000117.jpg
imagename:vehicle_0000117
vehicle
imagename:vehicle_0000117
plate
imagename:vehicle_0000117
vehicle
imagename:vehicle_0000117
plate
/media/yyp/DATA/Public_Datasets/BITVehicle/Annotations/vehicle_0000117.xml
/media/yyp/DATA/Public_Datasets/BITVehicle/JPEGImages/vehicle_0000119.jpg
imagename:vehicle_0000119
vehicle
imagename:vehicle_0000119
plate
/media/yyp/DATA/Public_Datasets/BITVehicle/Annotations/vehicle_0000119.xml
/media/yyp/DATA/Public_Datasets/BITVehicle/JPEGImages/vehicle_0000120.jpg
imagename:vehicle_0000120
vehicle
imagename:vehicle_0000120
plate
/media/yyp/DATA/Public_Datasets/BITVehicle/Annotations/vehicle_0000120.xml
/media/yyp/DATA/Public_Datasets/BITVehicle/JPEGImages/vehicle_0000121.jpg
imagename:vehicle_0000121
vehicle
imagename:vehicle_0000121
plate
/media/yyp/DATA/Public_Datasets/BITVehicle/Annotations/vehicle_0000121.xml
/media/yyp/DATA/Public_Datasets/BITVehicle/JPEGImages/vehicle_0000122.jpg


/media/yyp/DATA/Public_Datasets/BITVehicle/JPEGImages/vehicle_0000218.jpg
imagename:vehicle_0000218
vehicle
imagename:vehicle_0000218
plate
/media/yyp/DATA/Public_Datasets/BITVehicle/Annotations/vehicle_0000218.xml
/media/yyp/DATA/Public_Datasets/BITVehicle/JPEGImages/vehicle_0000223.jpg
imagename:vehicle_0000223
vehicle
imagename:vehicle_0000223
plate
/media/yyp/DATA/Public_Datasets/BITVehicle/Annotations/vehicle_0000223.xml
/media/yyp/DATA/Public_Datasets/BITVehicle/JPEGImages/vehicle_0000225.jpg
imagename:vehicle_0000225
vehicle
imagename:vehicle_0000225
plate
/media/yyp/DATA/Public_Datasets/BITVehicle/Annotations/vehicle_0000225.xml
/media/yyp/DATA/Public_Datasets/BITVehicle/JPEGImages/vehicle_0000229.jpg
imagename:vehicle_0000229
vehicle
imagename:vehicle_0000229
plate
/media/yyp/DATA/Public_Datasets/BITVehicle/Annotations/vehicle_0000229.xml
/media/yyp/DATA/Public_Datasets/BITVehicle/JPEGImages/vehicle_0000237.jpg
imagename:vehicle_0000237
vehicle
imagename:vehicle_0000237
plate


/media/yyp/DATA/Public_Datasets/BITVehicle/JPEGImages/vehicle_0000359.jpg
imagename:vehicle_0000359
vehicle
imagename:vehicle_0000359
plate
/media/yyp/DATA/Public_Datasets/BITVehicle/Annotations/vehicle_0000359.xml
/media/yyp/DATA/Public_Datasets/BITVehicle/JPEGImages/vehicle_0000360.jpg
imagename:vehicle_0000360
vehicle
imagename:vehicle_0000360
plate
/media/yyp/DATA/Public_Datasets/BITVehicle/Annotations/vehicle_0000360.xml
/media/yyp/DATA/Public_Datasets/BITVehicle/JPEGImages/vehicle_0000361.jpg
imagename:vehicle_0000361
vehicle
imagename:vehicle_0000361
plate
imagename:vehicle_0000361
vehicle
imagename:vehicle_0000361
plate
/media/yyp/DATA/Public_Datasets/BITVehicle/Annotations/vehicle_0000361.xml
/media/yyp/DATA/Public_Datasets/BITVehicle/JPEGImages/vehicle_0000363.jpg
imagename:vehicle_0000363
vehicle
imagename:vehicle_0000363
vehicle
imagename:vehicle_0000363
plate
/media/yyp/DATA/Public_Datasets/BITVehicle/Annotations/vehicle_0000363.xml
/media/yyp/DATA/Public_Datasets/BITVehic

/media/yyp/DATA/Public_Datasets/BITVehicle/JPEGImages/vehicle_0000471.jpg
imagename:vehicle_0000471
vehicle
imagename:vehicle_0000471
plate
/media/yyp/DATA/Public_Datasets/BITVehicle/Annotations/vehicle_0000471.xml
/media/yyp/DATA/Public_Datasets/BITVehicle/JPEGImages/vehicle_0000473.jpg
imagename:vehicle_0000473
vehicle
imagename:vehicle_0000473
plate
imagename:vehicle_0000473
vehicle
imagename:vehicle_0000473
plate
/media/yyp/DATA/Public_Datasets/BITVehicle/Annotations/vehicle_0000473.xml
/media/yyp/DATA/Public_Datasets/BITVehicle/JPEGImages/vehicle_0000481.jpg
imagename:vehicle_0000481
vehicle
imagename:vehicle_0000481
plate
imagename:vehicle_0000481
vehicle
imagename:vehicle_0000481
plate
imagename:vehicle_0000481
vehicle
imagename:vehicle_0000481
plate
/media/yyp/DATA/Public_Datasets/BITVehicle/Annotations/vehicle_0000481.xml
/media/yyp/DATA/Public_Datasets/BITVehicle/JPEGImages/vehicle_0000488.jpg
imagename:vehicle_0000488
vehicle
imagename:vehicle_0000488
plate
/media/yyp/DATA/P

...

/media/yyp/DATA/Public_Datasets/BITVehicle/JPEGImages/vehicle_0009047.jpg
imagename:vehicle_0009047
plate
imagename:vehicle_0009047
vehicle
/media/yyp/DATA/Public_Datasets/BITVehicle/Annotations/vehicle_0009047.xml
/media/yyp/DATA/Public_Datasets/BITVehicle/JPEGImages/vehicle_0009055.jpg
imagename:vehicle_0009055
plate
imagename:vehicle_0009055
vehicle
/media/yyp/DATA/Public_Datasets/BITVehicle/Annotations/vehicle_0009055.xml
/media/yyp/DATA/Public_Datasets/BITVehicle/JPEGImages/vehicle_0009056.jpg
imagename:vehicle_0009056
plate
imagename:vehicle_0009056
vehicle
/media/yyp/DATA/Public_Datasets/BITVehicle/Annotations/vehicle_0009056.xml
/media/yyp/DATA/Public_Datasets/BITVehicle/JPEGImages/vehicle_0009057.jpg
imagename:vehicle_0009057
plate
imagename:vehicle_0009057
vehicle
/media/yyp/DATA/Public_Datasets/BITVehicle/Annotations/vehicle_0009057.xml
/media/yyp/DATA/Public_Datasets/BITVehicle/JPEGImages/vehicle_0009059.jpg
imagename:vehicle_0009059
plate
imagename:vehicle_0009059
vehicle


/media/yyp/DATA/Public_Datasets/BITVehicle/JPEGImages/vehicle_0009219.jpg
imagename:vehicle_0009219
plate
imagename:vehicle_0009219
vehicle
/media/yyp/DATA/Public_Datasets/BITVehicle/Annotations/vehicle_0009219.xml
/media/yyp/DATA/Public_Datasets/BITVehicle/JPEGImages/vehicle_0009223.jpg
imagename:vehicle_0009223
plate
imagename:vehicle_0009223
vehicle
/media/yyp/DATA/Public_Datasets/BITVehicle/Annotations/vehicle_0009223.xml
/media/yyp/DATA/Public_Datasets/BITVehicle/JPEGImages/vehicle_0009230.jpg
imagename:vehicle_0009230
plate
imagename:vehicle_0009230
plate
imagename:vehicle_0009230
vehicle
imagename:vehicle_0009230
vehicle
/media/yyp/DATA/Public_Datasets/BITVehicle/Annotations/vehicle_0009230.xml
/media/yyp/DATA/Public_Datasets/BITVehicle/JPEGImages/vehicle_0009231.jpg
imagename:vehicle_0009231
plate
imagename:vehicle_0009231
vehicle
/media/yyp/DATA/Public_Datasets/BITVehicle/Annotations/vehicle_0009231.xml
/media/yyp/DATA/Public_Datasets/BITVehicle/JPEGImages/vehicle_0009232.jpg


/media/yyp/DATA/Public_Datasets/BITVehicle/JPEGImages/vehicle_0009371.jpg
imagename:vehicle_0009371
plate
imagename:vehicle_0009371
vehicle
/media/yyp/DATA/Public_Datasets/BITVehicle/Annotations/vehicle_0009371.xml
/media/yyp/DATA/Public_Datasets/BITVehicle/JPEGImages/vehicle_0009378.jpg
imagename:vehicle_0009378
plate
imagename:vehicle_0009378
vehicle
/media/yyp/DATA/Public_Datasets/BITVehicle/Annotations/vehicle_0009378.xml
/media/yyp/DATA/Public_Datasets/BITVehicle/JPEGImages/vehicle_0009382.jpg
imagename:vehicle_0009382
plate
imagename:vehicle_0009382
vehicle
/media/yyp/DATA/Public_Datasets/BITVehicle/Annotations/vehicle_0009382.xml
/media/yyp/DATA/Public_Datasets/BITVehicle/JPEGImages/vehicle_0009383.jpg
imagename:vehicle_0009383
plate
imagename:vehicle_0009383
vehicle
/media/yyp/DATA/Public_Datasets/BITVehicle/Annotations/vehicle_0009383.xml
/media/yyp/DATA/Public_Datasets/BITVehicle/JPEGImages/vehicle_0009387.jpg
imagename:vehicle_0009387
plate
imagename:vehicle_0009387
vehicle


/media/yyp/DATA/Public_Datasets/BITVehicle/JPEGImages/vehicle_0009524.jpg
imagename:vehicle_0009524
plate
imagename:vehicle_0009524
vehicle
/media/yyp/DATA/Public_Datasets/BITVehicle/Annotations/vehicle_0009524.xml
/media/yyp/DATA/Public_Datasets/BITVehicle/JPEGImages/vehicle_0009532.jpg
imagename:vehicle_0009532
plate
imagename:vehicle_0009532
vehicle
/media/yyp/DATA/Public_Datasets/BITVehicle/Annotations/vehicle_0009532.xml
/media/yyp/DATA/Public_Datasets/BITVehicle/JPEGImages/vehicle_0009535.jpg
imagename:vehicle_0009535
plate
imagename:vehicle_0009535
vehicle
/media/yyp/DATA/Public_Datasets/BITVehicle/Annotations/vehicle_0009535.xml
/media/yyp/DATA/Public_Datasets/BITVehicle/JPEGImages/vehicle_0009538.jpg
imagename:vehicle_0009538
plate
imagename:vehicle_0009538
vehicle
/media/yyp/DATA/Public_Datasets/BITVehicle/Annotations/vehicle_0009538.xml
/media/yyp/DATA/Public_Datasets/BITVehicle/JPEGImages/vehicle_0009541.jpg
imagename:vehicle_0009541
plate
imagename:vehicle_0009541
vehicle


/media/yyp/DATA/Public_Datasets/BITVehicle/JPEGImages/vehicle_0009686.jpg
imagename:vehicle_0009686
vehicle
imagename:vehicle_0009686
plate
/media/yyp/DATA/Public_Datasets/BITVehicle/Annotations/vehicle_0009686.xml
/media/yyp/DATA/Public_Datasets/BITVehicle/JPEGImages/vehicle_0009687.jpg
imagename:vehicle_0009687
vehicle
imagename:vehicle_0009687
plate
/media/yyp/DATA/Public_Datasets/BITVehicle/Annotations/vehicle_0009687.xml
/media/yyp/DATA/Public_Datasets/BITVehicle/JPEGImages/vehicle_0009701.jpg
imagename:vehicle_0009701
vehicle
imagename:vehicle_0009701
plate
/media/yyp/DATA/Public_Datasets/BITVehicle/Annotations/vehicle_0009701.xml
/media/yyp/DATA/Public_Datasets/BITVehicle/JPEGImages/vehicle_0009706.jpg
imagename:vehicle_0009706
vehicle
imagename:vehicle_0009706
plate
/media/yyp/DATA/Public_Datasets/BITVehicle/Annotations/vehicle_0009706.xml
/media/yyp/DATA/Public_Datasets/BITVehicle/JPEGImages/vehicle_0009708.jpg
imagename:vehicle_0009708
vehicle
imagename:vehicle_0009708
plate
